# 第十九章 动态属性和特性

## 19.1使用动态属性转换数据

In [70]:
from urllib.request import urlopen
import warnings
import os
import json

URL="http://www.oreilly.com/pub/sc/osconfeed"
JSON='data/osconfeed.json'

def load():
    if not os.path.exists(JSON):
        msg='downloading {} to {}'.format(URL,JSON)
        warnings.warn(msg)
        with urlopen(URL) as remote,open(JSON,'wb') as local:
            local.write(remote.read())
    
    with open(JSON,encoding='utf8') as fp:
        return json.load(fp)

In [2]:
os.chdir(r'D:\WORKSPACE2\python35\python8.25\fluent_python')
os.getcwd()

'D:\\WORKSPACE2\\python35\\python8.25\\fluent_python'

In [3]:
feed=load()

In [9]:
type(feed)

dict

In [4]:
sorted(feed['Schedule'].keys())

['conferences', 'events', 'speakers', 'venues']

In [6]:
for key,value in sorted(feed['Schedule'].items()):
    print('{:3} {}'.format(len(value),key))

  1 conferences
494 events
357 speakers
 53 venues


In [7]:
feed['Schedule']['speakers'][-1]['name']

'Carina C. Zona'

### 19.1.1 使用动态属性访问json类数据

In [8]:
#explore0.py:把一个JSON数据集转换成一个嵌套FrozenJson对象、列表和简单类型的FrozenJSON对象

from collections import abc

class FrozenJSON:
    
    def __init__(self,mapping):
        self.__data=dict(mapping)
    
    def __getattr__(self, name):
        if hasattr(self.__data,name):
            return getattr(self.__data,name)
        
        else:
            return  FrozenJSON.build(self.__data[name])
    @classmethod
    def build(cls,obj):
        if isinstance(obj,abc.Mapping):
            return cls(obj)
        elif isinstance(obj,abc.MutableSequence):
            return [cls.build(item) for item in obj]
        else:
            return obj
        

In [21]:
raw_feed = load()
feed=FrozenJSON(raw_feed)
print(type(raw_feed))
feed

<class 'dict'>


In [11]:
len(feed.Schedule.speakers)

357

In [22]:
sorted(feed.Schedule.keys())

['conferences', 'events', 'speakers', 'venues']

In [23]:
for key,value in sorted(feed.Schedule.items()):
    print('{:3} {}'.format(len(value),key))

  1 conferences
494 events
357 speakers
 53 venues


In [24]:
feed.Schedule.speakers[-1].name

'Carina C. Zona'

In [25]:
talk=feed.Schedule.events[40]

In [26]:
type(talk)

__main__.FrozenJSON

In [27]:
talk.name

'There *Will* Be Bugs'

In [28]:
talk.speakers

[3471, 5199]

In [29]:
talk.flavor #读取不存在的属性会抛出KeyError异常，而不是通常抛出的AttributeError异常

KeyError: 'flavor'

### 19.1.2处理无效属性名

FrozenJSON类有个缺陷，没有对名称为python关键字的属性做特殊处理。

In [30]:
grad=FrozenJSON({'name':'Jim bo','class':1982})

In [31]:
grad.class

SyntaxError: invalid syntax (<ipython-input-31-bb5c99ef29c5>, line 1)

In [32]:
#当然，可以这么做
getattr(grad,'class')

1982

In [48]:
#最好的方法是传给FrozenJSON.__init__方法的映射中是否有键的名称为关键字，如果有，那么在键名后加上_.
class FrozenJSON:
    """A read-only façade for navigating a JSON-like object
       using attribute notation
    """

# BEGIN EXPLORE1
    def __init__(self, mapping):
        self.__data = {}
        for key, value in mapping.items():
            if keyword.iskeyword(key):  # <1>
                key += '_'
            self.__data[key] = value
# END EXPLORE1

    def __getattr__(self, name):
        if hasattr(self.__data, name):
            return getattr(self.__data, name)
        else:
            return FrozenJSON.build(self.__data[name])

    @classmethod
    def build(cls, obj):
        if isinstance(obj, abc.Mapping):
            return cls(obj)
        elif isinstance(obj, abc.MutableSequence):
            return [cls.build(item) for item in obj]
        else:  # <8>
            return obj

In [45]:
grad=FrozenJSON({'name':'Jim bo','class':1982})
grad.class_

1982

类方法build把嵌套结构转换成FrozenJSON实例或FrozenJSON实力列表，因此__getatrr__方法使用这个方法访问属性时，能为不同的值返回不同类型的对象

### 19.1.3 使用__new__方法以灵活的方式创建对象

构建实例的特殊方法__new__，这是个类方法（使用特殊方式处理，因此不用使用@classmethod装饰器），必须返回一个实例

In [50]:
#用__new__方法取代build方法，构建可能是也可能不是FrozenJSON实例的新对象
from collections import abc
class FrozenJSON:
    def __new__(cls, arg):
        if isinstance(arg,abc.Mapping):
            #默认的行为是委托给超类的__new__方法。这里调用的是objec基类的__new__方法，把唯一的参数设为FrozenJSON
            #super().__new__(cls)表达式会调用object.__new__(FrozenJSON),而object类构建的实例实际上是FrozenJSON实例
            return super().__new__(cls)
        elif isinstance(arg,abc.MutableSequence):
            return [cls(item) for item in arg]
        else:
            return arg
        
    def __init__(self, mapping):
        self.__data = {}
        for key, value in mapping.items():
            if keyword.iskeyword(key):  # <1>
                key += '_'
            self.__data[key] = value


    def __getattr__(self, name):
        if hasattr(self.__data, name):
            return getattr(self.__data, name)
        else:
            return FrozenJSON(self.__data[name])      
        
        

In [53]:
grad=FrozenJSON({'name':[{'subname':'Jim bo',},'im','ct'],'class':1982})
grad.name

[<__main__.FrozenJSON at 0x1e15c230860>, 'im', 'ct']

In [56]:
grad.name[0]

In [55]:
grad.name[0].subname

'Jim bo'

### 19.1.4 使用shelve模块调整OSCON数据源的结构

In [57]:
import sys
sys.path.insert(0,r'D:\WORKSPACE2\python35\python8.25\fluent_python')

In [80]:
#schedule.py：访问保存在shelve.Shelf对象的OSCON日程数据
import warnings

DB_NAME = 'data/schedule1_db'
CONFERENCE = 'conference.115'

class Record:
    def __init__(self,**kwargs):
        self.__dict__.update(kwargs) #使用关键字参数传入的属性构建实例的简便方式
    
def load_db(db):
    raw_data = load()#之前定义的方法
    warnings.warn('LOADING '+DB_NAME)
    for collection,rec_list in raw_data['Schedule'].items():
        record_type= collection[:-1] #record_type的值是去掉尾部's'后的集合名（即把'event'变成'event')
        for record in rec_list:
            key = '{}.{}'.format(record_type,record['serial'])#把record_type和serial字段构成key
            record['serial'] = key #把'serial'字段的值设为完整的键
            db[key]=Record(**record)

In [81]:
#测试schedule.py脚本
import shelve
db=shelve.open(DB_NAME)#打开或新建一个数据库文件
if CONFERENCE not in db:
    load_db(db)
    

c:\program files\python35\lib\site-packages\ipykernel_launcher.py:13: UserWarning: LOADING data/schedule1_db
  del sys.path[0]


In [82]:
speaker=db['speaker.3471']

In [83]:
type(speaker)

__main__.Record

In [84]:
speaker.name,speaker.twitter

('Anna Martelli Ravenscroft', 'annaraven')

In [85]:
db.close()#一定要记得关闭，可以使用with

In [86]:
db

### 19.1.5 使用特性获取链接的记录

In [94]:
import warnings
import inspect

DB_NAME='data/schedule2_db'
CONFERENCE='conference.115'

class Record:
    def __init__(self,**kwargs):
        self.__dict__.update(kwargs)
    
    def __eq__(self, other):
        if isinstance(other,Record):
            return self.__dict__ == other.__dict__
        else:
            return  NotImplemented

class MissingDatabaseError(RuntimeError):
    '''需要数据库但是没有指定数据库时抛出'''
    
class DbRecord(Record):
    __db = None
    
    @staticmethod
    def set_db(db):
        DbRecord.__db = db
        
    @staticmethod
    def get_db():
        return DbRecord.__db
    
    @classmethod
    def fetch(cls,ident):
        db=cls.get_db()
        try:
            return db[ident]
        except TypeError:
            if db is None:
                msg="database not set;call '{}.set_db(my_db)'"
                raise MissingDatabaseError(msg.format(cls.__name__))
            else:
                raise 
    def __repr__(self):
        if hasattr(self,'serial'):
            cls_name=self.__class__.__name__
            return '<{} serial={!r}>'.format(cls_name,self.serial)
        else:
            return super().__repr__()
            

In [95]:
class Event(DbRecord):
    
    @property
    def venue(self):
        key='venue.{}'.format(self.venue_serial)
        return self.__class__.fetch(key)
    
    @property
    def speakers(self):
        if not hasattr(self,'_speaker_objs'):
            spkr_serials = self.__dict__['speakers']
            fetch = self.__class__.fetch
            self._speaker_objs = [fetch('speaker.{}'.format(key)) for key in spkr_serials]
        return self._speaker_objs
    
    def __repr__(self):
        if hasattr(self,'name'):
            cls_name=self.__class__.__name__
            return '<{} {!r}>'.format(cls_name,self.name)
        else:
            return super().__repr__()

In [98]:
def load_db(db):
    raw_data=load()
    warnings.warn('loading'+DB_NAME)
    for collection,rec_list in raw_data['Schedule'].items():
        record_type = collection[:-1]
        cls_name = record_type.capitalize()
        cls=globals().get(cls_name,DbRecord)
        if inspect.isclass(cls) and issubclass(cls,DbRecord):
            factory = cls
        else:
            factory=DbRecord
        for record in rec_list:
            key = '{}.{}'.format(record_type,record['serial'])
            record['serial']=key
            db[key]=factory(**record)
            

In [99]:
db = shelve.open(DB_NAME)
if CONFERENCE not in db: load_db(db)
#db是一个字典类似的结构，{
# 'event.15313':Event实例({'serial':Event.15313,'name':'why',......}),
# (如果有Speaker子类)'speakers.3136':Speaker实例({'serial':'Speaker.3136','name':...}),
#...

DbRecord.set_db(db)

c:\program files\python35\lib\site-packages\ipykernel_launcher.py:3: UserWarning: loadingdata/schedule2_db
  This is separate from the ipykernel package so we can avoid doing imports until


In [101]:
event=DbRecord.fetch('event.33950')

In [102]:
event

<Event 'There *Will* Be Bugs'>

In [103]:
event.venue

<DbRecord serial='venue.1449'>

In [104]:
event.venue.name

'Portland 251'

In [105]:
for spkr in event.speakers:
    print('{0.serial}:{0.name}'.format(spkr))

speaker.3471:Anna Martelli Ravenscroft
speaker.5199:Alex Martelli


## 19.2使用特性验证属性

### 19.2.1 LineItem类第1版：表示订单中商品的类

In [106]:
class LineItem:

    def __init__(self, description, weight, price):
        self.description = description
        self.weight = weight
        self.price = price

    def subtotal(self):
        return self.weight * self.price
# END LINEITEM_V1

In [107]:
raisins = LineItem('Golden raisins', 10, 6.95)

In [108]:
raisins.subtotal()

69.5

In [109]:
raisins.weight=-20
raisins.subtotal()#金额为负值！

-139.0

### 19.2.2 LineItem类第2版：能验证值得特性

In [110]:
class LineItem:

    def __init__(self, description, weight, price):
        self.description = description
        self.weight = weight
        self.price = price

    def subtotal(self):
        return self.weight * self.price
    
    @property
    def weight(self):#实现特性的方法，其名称与公开属性名称一样——weight
        return  self.__weight #真正的值存储在__weight中
    @weight.setter#把读值方法和设值方法绑定在一起
    def weight(self,value):
        if value>0:
            self.__weight=value
        else:
            raise ValueError('value must be >0')

In [112]:
walnuts = LineItem('walnuts', 0, 10.00)

ValueError: value must be >0

In [113]:
walnuts = LineItem('walnuts', 5, 10.00)

In [114]:
walnuts.weight

5

![chapter19-2](image/chapter19-2.png)

walnuts = LineItem('walnuts', 5, 10.00)，当初始化到self.weight=5时，会调用weight的设置方法

## 19.3特性全解析

property其实是一个类，property构造方法如下：
property(fget=None,fset=None,fdel=None,doc=None)

In [115]:
#与上例相同，只不过没使用装饰器
class LineItem:

    def __init__(self, description, weight, price):
        self.description = description
        self.weight = weight
        self.price = price

    def subtotal(self):
        return self.weight * self.price

    def get_weight(self):  # <1>
        return self.__weight

    def set_weight(self, value):  # <2>
        if value > 0:
            self.__weight = value
        else:
            raise ValueError('value must be > 0')

    weight = property(get_weight, set_weight)  # 构建prope对象，然后赋值给公开的类属性


### 19.3.1特性会覆盖实例属性

In [116]:
class Class:
    data = 'the class data attr'
    @property
    def prop(self):
        return 'the prop value'

下面演示实例属性会覆盖类属性

In [117]:
obj=Class()

In [118]:
vars(obj)

{}

In [119]:
obj.data

'the class data attr'

In [119]:
obj.data='bar'

In [122]:
vars(obj)

{'data': 'bar'}

In [123]:
obj.data

'bar'

In [124]:
Class.data

'the class data attr'

下面演示实例属性不会覆盖类特性

In [125]:
Class.prop#得到的是特性对象本身,不会运行特性的读值方法

In [126]:
obj.prop#运行特性的读值方法

'the prop value'

In [127]:
obj.prop='foo'#尝试设置prop实例属性，失败

AttributeError: can't set attribute

In [128]:
obj.__dict__['prop']='foo'#但可以直接把'prop;存入obj.__dict__中

In [129]:
vars(obj)

{'data': 'bar', 'prop': 'foo'}

In [130]:
obj.prop#然而读取prop时仍会运行特性的读值方法。特性没有被实例属性覆盖

'the prop value'

In [130]:
Class.prop='baz'#覆盖Class.prop特性，销毁特性对象

In [132]:
obj.prop#obj.prop获取的是实例属性。Class.prop不是特性了

'foo'

新添加的类特性覆盖现有的实例属性

In [133]:
obj.data

'bar'

In [134]:
Class.data

'the class data attr'

In [135]:
Class.data=property(lambda self:'the "data" prop value')

In [137]:
obj.data

'the "data" prop value'

In [138]:
del Class.data

In [139]:
obj.data

'bar'

### 19.3.2特性的文档

## 19.4定义一个特性工厂函数

In [139]:
#quantity特性工厂函数的实现
def quantity(storage_name):
    
    def qty_getter(instance):
        return instance.__dict__[storage_name]#引用lestorage_name,闭包
    
    def qty_setter(instance,value):
        if value>0:
            instance.__dict__[storage_name]=value
        else:
            raise ValueError('value must be >0')
    return property(qty_getter,qty_setter)


In [141]:
class LineItem:
    weight=quantity('weight') #赋值语句右边先计算，因此使用quantity()时，weight类属性还不存在
    price=quantity('price')
    
    def __init__(self,description,weight,price):
        self.description=description
        self.weight=weight
        self.price=price
    
    def subtotal(self):
        return self.weight*self.price
    

In [142]:
walnuts = LineItem('walnuts', 0, 10.00)

ValueError: value must be >0

In [142]:
walnuts = LineItem('walnuts', 5, 10.00)

In [145]:
walnuts.weight

5

In [146]:
walnuts.subtotal()

50.0

In [147]:
sorted(vars(walnuts))

['description', 'price', 'weight']

In [148]:
sorted(vars(walnuts).items())

[('description', 'walnuts'), ('price', 10.0), ('weight', 5)]

注意，工厂函数构建的特性利用了之前说的行为：weight特性覆盖了weight实例属性，因此对self.weight或walnuts.weight的引用都由特性函数处理，只有直接存取__dict__属性才能跳过特性的处理逻辑

## 19.5处理属性删除操作

In [164]:
class BlackKnight:

    def __init__(self):
        self.members = ['an arm', 'another arm',
                        'a leg', 'another leg']
        self.phrases = ["'Tis but a scratch.",
                        "It's just a flesh wound.",
                        "I'm invincible!",
                        "All right, we'll call it a draw."]

    @property
    def member(self):
        print('next member is:')
        return self.members[0]

    @member.deleter
    def member(self):
        text = 'BLACK KNIGHT (loses {})\n-- {}'
        print(text.format(self.members.pop(0), self.phrases.pop(0)))
# END BLACK_KNIGHT

In [167]:
knight=BlackKnight()
knight.member

next member is:


In [168]:
del knight.member

BLACK KNIGHT (loses an arm)
-- 'Tis but a scratch.


In [169]:
del knight.member

BLACK KNIGHT (loses another arm)
-- It's just a flesh wound.


In [170]:
del knight.member

In [171]:
del knight.member

BLACK KNIGHT (loses another leg)
-- All right, we'll call it a draw.


不使用装饰器的句法：
member=property(member_getter,fdel=member.deleter

## 19.6处理属性的重要属性和函数

### 19.6.1影响属性处理方式的特殊属性

In [172]:
'''
__class__

__dict__

__slot__
'''

'\n__class__\n\n__dict__\n\n__slot__\n'